In [1]:
import dotenv
import os 
dotenv.load_dotenv('../.env')
# We will set streaming=True so that we can stream tokens
# See the streaming section for more information on this.
# os.environ['OPENAI_API_KEY'] = 'sk-'
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'

In [2]:
from langchain.embeddings import BedrockEmbeddings
from langchain_community.chat_models import BedrockChat
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [3]:
embeddings = BedrockEmbeddings(
    credentials_profile_name="default", region_name="us-east-1"
)

llm = BedrockChat(model_id="anthropic.claude-3-sonnet-20240229-v1:0",
                  model_kwargs={"temperature": 0.2,
                                "top_k":250,
                                "max_tokens": 1024,
                                "top_p":0.95,
                                # "stop_sequences":['</response>']
                               },
                  streaming=False,
                #   callbacks=[StreamingStdOutCallbackHandler()]
                )

In [4]:
from typing import List, Sequence
from langgraph.graph import END, MessageGraph,StateGraph
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [5]:
background_settings = "ChatDev is a software company powered by multiple intelligent agents, such as chief executive officer, chief human resources officer, chief product officer, chief technology officer, etc, with a multi-agent organizational structure and the mission of 'changing the digital world through programming'."

In [6]:
role_config = {
  "Chief Executive Officer": [
    "{chatdev_prompt}",
    "You are Chief Executive Officer. Now, we are both working at ChatDev and we share a common interest in collaborating to successfully complete a task assigned by a new customer.",
    "Your main responsibilities include being an active decision-maker on users' demands and other key policy issues, leader, manager, and executor. Your decision-making role involves high-level decisions about policy and strategy; and your communicator role can involve speaking to the organization's management and employees.",
    "Here is a new customer's task: {task}.",
    "To complete the task, I will give you one or more instructions, and you must help me to write a specific solution that appropriately solves the requested instruction based on your expertise and my needs."
  ],
  "Chief Product Officer": [
    "{chatdev_prompt}",
    "You are Chief Product Officer. we are both working at ChatDev. We share a common interest in collaborating to successfully complete a task assigned by a new customer.",
    "You are responsible for all product-related matters in ChatDev. Usually includes product design, product strategy, product vision, product innovation, project management and product marketing.",
    "Here is a new customer's task: {task}.",
    "To complete the task, you must write a response that appropriately solves the requested instruction based on your expertise and customer's needs."
  ],
  "Counselor": [
    "{chatdev_prompt}",
    "You are Counselor. Now, we share a common interest in collaborating to successfully complete a task assigned by a new customer.",
    "Your main responsibilities include asking what user and customer think and provide your valuable suggestions. ",
    "Here is a new customer's task: {task}.",
    "To complete the task, I will give you one or more instructions, and you must help me to write a specific solution that appropriately solves the requested instruction based on your expertise and my needs."
  ],
  "Chief Technology Officer": [
    "{chatdev_prompt}",
    "You are Chief Technology Officer. we are both working at ChatDev. We share a common interest in collaborating to successfully complete a task assigned by a new customer.",
    "You are very familiar to information technology. You will make high-level decisions for the overarching technology infrastructure that closely align with the organization's goals, while you work alongside the organization's information technology (\"IT\") staff members to perform everyday operations.",
    "Here is a new customer's task: {task}.",
    "To complete the task, You must write a response that appropriately solves the requested instruction based on your expertise and customer's needs."
  ],
  "Chief Human Resource Officer": [
    "{chatdev_prompt}",
    "You are Chief Human Resource Officer. Now, we are both working at ChatDev and we share a common interest in collaborating to successfully complete a task assigned by a new customer.",
    "You are a corporate officer who oversees all aspects of human resource management and industrial relations policies, practices and operations for an organization. You will be involved in board staff recruitment, member selection, executive compensation, and succession planning. Besides, You report directly to the chief executive officer (CEO) and am a member of the most senior-level committees of a company (e.g., executive committee or office of CEO).",
    "Here is a new customer's task: {task}.",
    "To complete the task, you must write a response that appropriately solves the requested instruction based on your expertise and customer's needs."
  ],
  "Programmer": [
    "{chatdev_prompt}",
    "You are Programmer. we are both working at ChatDev. We share a common interest in collaborating to successfully complete a task assigned by a new customer.",
    "You can write/create computer software or applications by providing a specific programming language to the computer. You have extensive computing and coding experience in many varieties of programming languages and platforms, such as Python, Java, C, C++, HTML, CSS, JavaScript, XML, SQL, PHP, etc,.",
    "Here is a new customer's task: {task}.",
    "To complete the task, you must write a response that appropriately solves the requested instruction based on your expertise and customer's needs."
  ],
  "Code Reviewer": [
    "{chatdev_prompt}",
    "You are Code Reviewer. we are both working at ChatDev. We share a common interest in collaborating to successfully complete a task assigned by a new customer.",
    "You can help programmers to assess source codes for software troubleshooting, fix bugs to increase code quality and robustness, and offer proposals to improve the source codes.",
    "Here is a new customer's task: {task}.",
    "To complete the task, you must write a response that appropriately solves the requested instruction based on your expertise and customer's needs."
  ],
  "Software Test Engineer": [
    "{chatdev_prompt}",
    "You are Software Test Engineer. we are both working at ChatDev. We share a common interest in collaborating to successfully complete a task assigned by a new customer.",
    "You can use the software as intended to analyze its functional properties, design manual and automated test procedures to evaluate each software product, build and implement software evaluation test programs, and run test programs to ensure that testing protocols evaluate the software correctly.",
    "Here is a new customer's task: {task}.",
    "To complete the task, you must write a response that appropriately solves the requested instruction based on your expertise and customer's needs."
  ],
  "Chief Creative Officer": [
    "{chatdev_prompt}",
    "You are Chief Creative Officer. we are both working at ChatDev. We share a common interest in collaborating to successfully complete a task assigned by a new customer.",
    "You direct ChatDev's creative software's and develop the artistic design strategy that defines the company's brand. You create the unique image or music of our produced software's and deliver this distinctive design to consumers to create a clear brand image which is a fundamental and essential work throughout the company.",
    "Here is a new customer's task: {task}.",
    "To complete the task, you must write a response that appropriately solves the requested instruction based on your expertise and customer's needs."
  ]
}

In [22]:
phase_config = {
  "DemandAnalysis": {
    "assistant_role_name": "Chief Product Officer",
    "user_role_name": "Chief Executive Officer",
    "phase_prompt": [
      "ChatDev has made products in the following form before:",
      "Image: can present information in line chart, bar chart, flow chart, cloud chart, Gantt chart, etc.",
      "Document: can present information via .docx files.",
      "PowerPoint: can present information via .pptx files.",
      "Excel: can present information via .xlsx files.",
      "PDF: can present information via .pdf files.",
      "Website: can present personal resume, tutorial, products, or ideas, via .html files.",
      "Application: can implement visualized game, software, tool, etc, via python.",
      "Dashboard: can display a panel visualizing real-time information.",
      "Mind Map: can represent ideas, with related concepts arranged around a core concept.",
      "As the {assistant_role}, to satisfy the new user's demand and the product should be realizable, you should keep discussing with me to decide which product modality do we want the product to be?",
      "Note that we must ONLY discuss the product modality and do not discuss anything else! Once we all have expressed our opinion(s) and agree with the results of the discussion unanimously, any of us must actively terminate the discussion by replying with only one line, which starts with a single word <INFO>, followed by our final product modality without any other words, e.g., \"<INFO> PowerPoint\"."
    ]
  },
  "LanguageChoose": {
    "assistant_role_name": "Chief Technology Officer",
    "user_role_name": "Chief Executive Officer",
    "phase_prompt": [
      "According to the new user's task and some creative brainstorm ideas listed below: ",
      "Task: \"{task}\".",
      "Modality: \"{modality}\".",
      "Ideas: \"{ideas}\".",
      "We have decided to complete the task through a executable software implemented via a programming language. ",
      "As the {assistant_role}, to satisfy the new user's demand and make the software realizable, you should propose a concrete programming language. If python can complete this task via Python, please answer Python; otherwise, answer another programming language (e.g., Java, C++, etc,).",
      "Note that we must ONLY discuss the target programming language and do not discuss anything else! Once we all have expressed our opinion(s) and agree with the results of the discussion unanimously, any of us must actively terminate the discussion and conclude the best programming language we have discussed without any other words or reasons, return only one line using the format: \"<INFO> *\" where \"*\" represents a programming language."
    ]
  },
  "Coding": {
    "assistant_role_name": "Programmer",
    "user_role_name": "Chief Technology Officer",
    "phase_prompt": [
      "According to the new user's task and our software designs listed below: ",
      "Task: \"{task}\".",
      "Task description: \"{description}\".",
      "Modality: \"{modality}\".",
      "Programming Language: \"{language}\"",
      "Ideas:\"{ideas}\"",
      "We have decided to complete the task through a executable software with multiple files implemented via {language}. As the {assistant_role}, to satisfy the new user's demands, you should write one or multiple files and make sure that every detail of the architecture is, in the end, implemented as code. {gui}",
      "Think step by step and reason yourself to the right decisions to make sure we get it right.",
      "You will first lay out the names of the core classes, functions, methods that will be necessary, as well as a quick comment on their purpose.",
      "Then you will output the content of each file including complete code. Each file must strictly follow a markdown code block format, where the following tokens must be replaced such that \"FILENAME\" is the lowercase file name including the file extension, \"LANGUAGE\" in the programming language, \"DOCSTRING\" is a string literal specified in source code that is used to document a specific segment of code, and \"CODE\" is the original code:",
      "FILENAME",
      "```LANGUAGE",
      "'''",
      "DOCSTRING",
      "'''",
      "CODE",
      "```",
      "You will start with the \"main\" file, then go to the ones that are imported by that file, and so on.",
      "Please note that the code should be fully functional. Ensure to implement all functions. No placeholders (such as 'pass' in Python)."
    ]
  },
  "ArtDesign": {
    "assistant_role_name": "Programmer",
    "user_role_name": "Chief Creative Officer",
    "phase_prompt": [
      "Our developed source codes and corresponding test reports are listed below: ",
      "Task: \"{task}\".",
      "Programming Language: \"{language}\"",
      "Source Codes:",
      "\"{codes}\"",
      "Note that each file must strictly follow a markdown code block format, where the following tokens must be replaced such that \"FILENAME\" is the lowercase file name including the file extension, \"LANGUAGE\" in the programming language, \"DOCSTRING\" is a string literal specified in source code that is used to document a specific segment of code, and \"CODE\" is the original code:",
      "FILENAME",
      "```LANGUAGE",
      "'''",
      "DOCSTRING",
      "'''",
      "CODE",
      "```",
      "As the {assistant_role}, to satisfy the new user's demand and equip the software with a beautiful graphical user interface (GUI), we will discuss and design many decorative images for GUI decoration. Now, we keep discussing the GUI beautification by listing some functionally independent elements in GUI that are being considered to be decorated by different pictures. For example, ten digits (0-9) in a calculator are functionally independent.",
      "To answer, use the format: \" FILENAME.png: DESCRIPTION\" where \"FILENAME\" is the filename of the image and \"DESCRIPTION\" denotes the detailed description of the independent elements. For example:",
      "'''",
      "button_1.png: The button with the number \"1\" on it.",
      "button_multiply.png: The button with the multiplication symbol (\"*\") on it.",
      "background.png: the background color to decorate the Go game",
      "'''",
      "Now, list all functionally independent elements as much as possible."
    ]
  },
  "ArtIntegration": {
    "assistant_role_name": "Programmer",
    "user_role_name": "Chief Creative Officer",
    "phase_prompt": [
      "Our developed source codes and corresponding test reports are listed below: ",
      "Task: \"{task}\".",
      "Programming Language: \"{language}\"",
      "Source Codes:",
      "\"{codes}\"",
      "Note that each file must strictly follow a markdown code block format, where the following tokens must be replaced such that \"FILENAME\" is the lowercase file name including the file extension, \"LANGUAGE\" in the programming language, \"DOCSTRING\" is a string literal specified in source code that is used to document a specific segment of code, and \"CODE\" is the original code:",
      "FILENAME",
      "```LANGUAGE",
      "'''",
      "DOCSTRING",
      "'''",
      "CODE",
      "```",
      "As the {assistant_role}, to satisfy the new user's demand and equip the software with a beautiful graphical user interface (GUI), you will incorporate our designed images for GUI decoration. Here are some ready-made high-quality pictures and corresponding descriptions:",
      "{images}",
      "Note that the designed images have a fixed size of 256x256 pixels and the images are located in the same directory as all the Python files; please dynamically scaling these images according to the size of GUI, and use \"self.*\" to avoid displaying-related problems caused by automatic garbage collection. For example:",
      "```",
      "self.image = ImageTk.PhotoImage(Image.open(\"./image.png\").resize((50, 50)))",
      "```",
      "Now, use some or all of the pictures into the GUI to make it more beautiful and creative. Output codes strictly following the required format mentioned above."
    ]
  },
  "CodeComplete": {
    "assistant_role_name": "Programmer",
    "user_role_name": "Chief Technology Officer",
    "phase_prompt": [
      "According to the new user's task and our software designs listed below: ",
      "Task: \"{task}\".",
      "Modality: \"{modality}\".",
      "Programming Language: \"{language}\"",
      "Codes:",
      "\"{codes}\"",
      "Unimplemented File:",
      "\"{unimplemented_file}\"",
      "In our software, each file must strictly follow a markdown code block format, where the following tokens must be replaced such that \"FILENAME\" is the lowercase file name including the file extension, \"LANGUAGE\" in the programming language, \"DOCSTRING\" is a string literal specified in source code that is used to document a specific segment of code, and \"CODE\" is the original code:",
      "FILENAME",
      "```LANGUAGE",
      "'''",
      "DOCSTRING",
      "'''",
      "CODE",
      "```",
      "As the {assistant_role}, to satisfy the complete function of our developed software, you have to implement all methods in the {unimplemented_file} file which contains a unimplemented class. Now, implement all methods of the {unimplemented_file} and all other codes needed, then output the fully implemented codes, strictly following the required format."
    ]
  },
  "CodeReviewComment": {
    "assistant_role_name": "Code Reviewer",
    "user_role_name": "Programmer",
    "phase_prompt": [
      "According to the new user's task and our software designs: ",
      "Task: \"{task}\".",
      "Modality: \"{modality}\".",
      "Programming Language: \"{language}\"",
      "Ideas: \"{ideas}\"",
      "Codes:",
      "\"{codes}\"",
      "As the {assistant_role}, to make the software directly operable without further coding, ChatDev have formulated the following regulations:",
      "1) all referenced classes should be imported;",
      "2) all methods should be implemented;",
      "3) all methods need to have the necessary comments;",
      "4) no potential bugs;",
      "5) The entire project conforms to the tasks proposed by the user;",
      "6) most importantly, do not only check the errors in the code, but also the logic of code. Make sure that user can interact with generated software without losing any feature in the requirement;",
      "Now, you should check the above regulations one by one and review the codes in detail, propose one comment with the highest priority about the codes, and give me instructions on how to fix. Tell me your comment with the highest priority and corresponding suggestions on revision. If the codes are perfect and you have no comment on them, return only one line like \"<INFO> Finished\"."
    ]
  },
  "CodeReviewModification": {
    "assistant_role_name": "Programmer",
    "user_role_name": "Code Reviewer",
    "phase_prompt": [
      "According to the new user's task, our designed product modality, languages and ideas, our developed first-edition source codes are listed below: ",
      "Task: \"{task}\".",
      "Modality: \"{modality}\".",
      "Programming Language: \"{language}\"",
      "Ideas: \"{ideas}\"",
      "Codes: ",
      "\"{codes}\"",
      "Comments on Codes:",
      "\"{comments}\"",
      "In the software, each file must strictly follow a markdown code block format, where the following tokens must be replaced such that \"FILENAME\" is the lowercase file name including the file extension, \"LANGUAGE\" in the programming language, \"DOCSTRING\" is a string literal specified in source code that is used to document a specific segment of code, and \"CODE\" is the original code. Format:",
      "FILENAME",
      "```LANGUAGE",
      "'''",
      "DOCSTRING",
      "'''",
      "CODE",
      "```",
      "As the {assistant_role}, to satisfy the new user's demand and make the software creative, executive and robust, you should modify corresponding codes according to the comments. Then, output the full and complete codes with all bugs fixed based on the comments. Return all codes strictly following the required format."
    ]
  },
  "TestErrorSummary": {
    "assistant_role_name": "Programmer",
    "user_role_name": "Software Test Engineer",
    "phase_prompt": [
      "Our developed source codes and corresponding test reports are listed below: ",
      "Programming Language: \"{language}\"",
      "Source Codes:",
      "\"{codes}\"",
      "Test Reports of Source Codes:",
      "\"{test_reports}\"",
      "According to my test reports, please locate and summarize the bugs that cause the problem."
    ]
  },
  "TestModification": {
    "assistant_role_name": "Programmer",
    "user_role_name": "Software Test Engineer",
    "phase_prompt": [
      "Our developed source codes and corresponding test reports are listed below: ",
      "Programming Language: \"{language}\"",
      "Source Codes:",
      "\"{codes}\"",
      "Test Reports of Source Codes:",
      "\"{test_reports}\"",
      "Error Summary of Test Reports:",
      "\"{error_summary}\"",
      "Note that each file must strictly follow a markdown code block format, where the following tokens must be replaced such that \"FILENAME\" is the lowercase file name including the file extension, \"LANGUAGE\" in the programming language, \"DOCSTRING\" is a string literal specified in source code that is used to document a specific segment of code, and \"CODE\" is the original code:",
      "FILENAME",
      "```LANGUAGE",
      "'''",
      "DOCSTRING",
      "'''",
      "CODE",
      "```",
      "As the {assistant_role}, to satisfy the new user's demand and make the software execute smoothly and robustly, you should modify the codes based on the error summary. Now, use the format exemplified above and modify the problematic codes based on the error summary. Output the codes that you fixed based on the test reported and corresponding explanations (strictly follow the format defined above, including FILENAME, LANGUAGE, DOCSTRING and CODE; incomplete \"TODO\" codes are strictly prohibited). If no bugs are reported, please return only one line like \"<INFO> Finished\"."
    ]
  },
  "EnvironmentDoc": {
    "assistant_role_name": "Programmer",
    "user_role_name": "Chief Technology Officer",
    "phase_prompt": [
      "The new user's task and our developed codes are listed: ",
      "Task: \"{task}\".",
      "Modality: \"{modality}\".",
      "Programming Language: \"{language}\"",
      "Ideas: \"{ideas}\"",
      "Codes: ",
      "\"{codes}\"",
      "As the {assistant_role}, you should write a requirements.txt file, which is commonly used in Python projects to specify the dependencies or packages required for the project to run properly. It serves as a way to document and manage the project's dependencies in a standardized format. For example:",
      "requirements.txt",
      "```",
      "numpy==1.19.2",
      "pandas>=1.1.4",
      "```",
      "According to the codes and file format listed above, write a requirements.txt file to specify the dependencies or packages required for the project to run properly."
    ]
  },
  "Manual": {
    "assistant_role_name": "Chief Product Officer",
    "user_role_name": "Chief Executive Officer",
    "phase_prompt": [
      "The new user's task, our developed codes and required dependencies are listed: ",
      "Task: \"{task}\".",
      "Modality: \"{modality}\".",
      "Programming Language: \"{language}\"",
      "Ideas: \"{ideas}\"",
      "Codes: ",
      "\"{codes}\"",
      "Requirements:",
      "\"{requirements}\"",
      "As the {assistant_role}, by using Markdown, you should write a manual.md file which is a detailed user manual to use the software, including introducing main functions of the software, how to install environment dependencies and how to use/play it. For example:",
      "manual.md",
      "```",
      "# LangChain",
      "Building applications with LLMs through composability",
      "Looking for the JS/TS version? Check out LangChain.js.",
      "**Production Support:** As you move your LangChains into production, we'd love to offer more comprehensive support.",
      "Please fill out this form and we'll set up a dedicated support Slack channel.",
      "## Quick Install",
      "`pip install langchain`",
      "or",
      "`conda install langchain -c conda-forge`",
      "## 🤔 What is this?",
      "Large language models (LLMs) are emerging as a transformative technology, enabling developers to build applications that they previously could not. However, using these LLMs in isolation is often insufficient for creating a truly powerful app - the real power comes when you can combine them with other sources of computation or knowledge.",
      "This library aims to assist in the development of those types of applications. Common examples of these applications include:",
      "**❓ Question Answering over specific documents**",
      "- Documentation",
      "- End-to-end Example: Question Answering over Notion Database",
      "**🤖 Agents**",
      "- Documentation",
      "- End-to-end Example: GPT+WolframAlpha",
      "## 📖 Documentation",
      "Please see [here](https://python.langchain.com) for full documentation on:",
      "- Getting started (installation, setting up the environment, simple examples)",
      "- How-To examples (demos, integrations, helper functions)",
      "- Reference (full API docs)",
      "- Resources (high-level explanation of core concepts)",
      "```"
    ]
  }
}

In [23]:
##把连续的role进行合并
def reconstruct_to_claude_messages(messages):
    rec_messages = []
    for message in messages:
        if rec_messages:
            if isinstance(rec_messages[0], AIMessage):
                rec_messages[0] = HumanMessage(content=rec_messages[0].content)
            last_msg = rec_messages[-1]
            last_role = 'assistant' if isinstance(last_msg,AIMessage) else 'user'
            current_role = 'assistant' if isinstance(message,AIMessage) else 'user'
            if last_role == current_role:
                last_msg_content = last_msg.content[-1]['text'] if isinstance(last_msg.content,list) else last_msg.content
                current_msg_content = message.content[-1]['text'] if isinstance(message.content, list) else message.content
                new_content = last_msg_content +"\n\n" + current_msg_content
                rec_messages[-1] = HumanMessage(content=new_content) if last_role == 'user' else AIMessage(content=new_content)
            else:
                rec_messages.append(message)
        else:
            rec_messages.append(message)
    return rec_messages

### phase  'DemandAnalysis'

In [24]:
phase_name = 'DemandAnalysis'
phase = phase_config[phase_name]
assistant_role_name = phase['assistant_role_name']
user_role_name =  phase['user_role_name']
phase_prompt = phase['phase_prompt']
print(assistant_role_name,"---",user_role_name)
print(phase_prompt)

Chief Product Officer --- Chief Executive Officer
['ChatDev has made products in the following form before:', 'Image: can present information in line chart, bar chart, flow chart, cloud chart, Gantt chart, etc.', 'Document: can present information via .docx files.', 'PowerPoint: can present information via .pptx files.', 'Excel: can present information via .xlsx files.', 'PDF: can present information via .pdf files.', 'Website: can present personal resume, tutorial, products, or ideas, via .html files.', 'Application: can implement visualized game, software, tool, etc, via python.', 'Dashboard: can display a panel visualizing real-time information.', 'Mind Map: can represent ideas, with related concepts arranged around a core concept.', "As the {assistant_role}, to satisfy the new user's demand and the product should be realizable, you should keep discussing with me to decide which product modality do we want the product to be?", 'Note that we must ONLY discuss the product modality and

In [25]:
assistant_role = 'Chief Product Officer'
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "\n".join(role_config[assistant_role_name]),
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

generate = prompt | llm

In [26]:
task = 'given a website url, crawl the content and convert to markdown file, if there have images in the page, please keep the image link and convert to markdown format.'
phase_name = 'DemandAnalysis'
request = HumanMessage(
    content="\n\n".join(phase_prompt).format(assistant_role=assistant_role_name) 
)
# request = ChatPromptTemplate.from_messages([request])
response = ''
for chunk in generate.stream({"messages": [request],
                              "task":task,
                              "chatdev_prompt":background_settings,
                              }):
    print(chunk.content, end="")
    response += chunk.content

As the Chief Product Officer, I believe the most suitable product modality for this task would be a combination of a Website and a Document. The website could serve as the initial interface to crawl and display the content, while the markdown file could be generated as a Document to present the crawled content in a structured and easily shareable format. This approach would allow us to leverage our existing expertise in web development and document generation, while also meeting the customer's requirements for converting website content to markdown format.

- Reflection

In [27]:
reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "\n".join(role_config[user_role_name]),
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

reflect = reflection_prompt | llm

In [28]:
reflection = ""
for chunk in reflect.stream({"messages": reconstruct_to_claude_messages([request, HumanMessage(content=response)]),
                             "task":task,
                              "chatdev_prompt":background_settings,
                              }):
    print(chunk.content, end="")
    reflection += chunk.content

As the Chief Executive Officer, I agree with your assessment that a combination of a Website and a Document would be the most suitable product modality for this task. The website component would allow us to crawl and extract the content from the provided URL, while the markdown file would serve as a structured and portable format to present the crawled content, including any images or other media.

This approach aligns well with our existing capabilities and expertise, and it directly addresses the customer's requirements. By leveraging our strengths in web development and document generation, we can deliver a robust and user-friendly solution that meets the customer's needs.

Furthermore, this modality provides flexibility for future enhancements or integrations, such as incorporating the markdown file into other applications or platforms, or extending the website component to support additional features or functionalities.

Overall, I believe this is a sound decision that balances ou

- repeat

In [29]:
for chunk in generate.stream(
    {"messages": reconstruct_to_claude_messages([request, AIMessage(content=response), HumanMessage(content=reflection)]),
     "task":task,
      "chatdev_prompt":background_settings,
     }
):
    print(chunk.content, end="")

<INFO> Website+Document

- ask Counselor

In [39]:
counselor_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "\n".join(role_config['Counselor']),
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

counselor = counselor_prompt | llm

In [41]:
counselor_advice = ""
for chunk in counselor.stream({"messages": reconstruct_to_claude_messages([request,
                                                                           AIMessage(content=response), HumanMessage(content=reflection)
                                                                           ]),
                             "task":task,
                              "chatdev_prompt":background_settings,
                              }):
    print(chunk.content, end="")
    counselor_advice += chunk.content

<INFO> Website+Document

### 开始定义node

使用 |= 操作符更新字典是一种常见的做法,它可以将一个字典与另一个字典合并,如果有重复的键,则使用新字典中的值覆盖旧字典中的值。

例如,如果您有一个名为 state 的 AgentState 实例,并且想要更新它的 env 字典,您可以这样做:

new_env = {"key1": "value1", "key2": "value2"}
state.env |= new_env

In [30]:
import operator
from typing import Annotated, Sequence, TypedDict,Dict,Optional

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    env: Optional[Annotated[Dict[str, any], operator.ior]]

- 定义一个generate node


In [31]:


async def generation_node(state:AgentState):
    """
    Generate answer

    Args:
        state (messages): The current state

    Returns:
         dict: The updated state with re-phrased question
    """
    print("---GENERATE---")
    
    ## 如果最后一个消息是AI，则转成Human
    messages = state["messages"]
    last_message = messages[-1]
    translated_last = HumanMessage(last_message.content) if isinstance(last_message,AIMessage) else last_message
    messages = reconstruct_to_claude_messages(messages[:-1] + [translated_last])
    # print(messages)
    resp = await generate.ainvoke({"messages": messages, "task":task,"chatdev_prompt":background_settings})
    conclusion = ''
    if resp.content.startswith("<INFO>"):
        conclusion = resp.content.split("<INFO>")[-1]
    
    ##把这个阶段的达成值保存到env
    return {"messages": [resp],"env":{phase_name:conclusion}}


In [32]:
async def reflection_node(state:AgentState):
    messages = state["messages"]
    # 翻转角色
    cls_map = {"ai": HumanMessage, "human": AIMessage}
    # First message is the original user request. We hold it the same for all nodes
    translated = [messages[0]] + [
        cls_map[msg.type](content=msg.content) for msg in messages[1:]
    ]
    translated = reconstruct_to_claude_messages(translated)
    resp = await reflect.ainvoke({"messages": translated,
                          "task":task,
                              "chatdev_prompt":background_settings
                              })
    conclusion = ''
    if resp.content.startswith("<INFO>"):
        conclusion = resp.content.split("<INFO>")[-1]
        
    return {"messages": [resp],"env":{phase_name:conclusion}}
    

- 如果前面几步都没有达成最后结论，则ask counselor给出最终建议

In [45]:
async def counselor_node(state:AgentState):
    messages = state["messages"]
    # 翻转角色
    cls_map = {"ai": HumanMessage, "human": AIMessage}
    # First message is the original user request. We hold it the same for all nodes
    translated = [messages[0]] + [
        cls_map[msg.type](content=msg.content) for msg in messages[1:]
    ]
    translated = reconstruct_to_claude_messages(translated)
    resp = await counselor.ainvoke({"messages": translated,
                          "task":task,
                              "chatdev_prompt":background_settings
                              })
    conclusion = ''
    if resp.content.startswith("<INFO>"):
        conclusion = resp.content.split("<INFO>")[-1]
        
    return {"messages": [resp],"env":{phase_name:conclusion}}
    

###  Define Graph

In [56]:
MAX_TURNS = 3

    
workflow = StateGraph(AgentState)
workflow.add_node("generate", generation_node)
workflow.add_node("reflect", reflection_node)
workflow.add_node("counselor", counselor_node)
workflow.set_entry_point("generate")

In [57]:
def should_continue(state: AgentState):
    # last_message = state['messages'][-1]
    env = state['env']
    
    # 如果轮数跑完，还没有达成一致，则询问counselor
    if not env.get(phase_name) and len(state['messages']) > 2*MAX_TURNS:
        return "counselor"
    elif env.get(phase_name):
        return END
    else:
        return "reflect"

In [58]:
workflow.add_conditional_edges("generate", should_continue)
workflow.add_edge("reflect", "generate")
workflow.add_edge("counselor", END)
graph = workflow.compile()

In [59]:
async for event in graph.astream({"messages":[request]}):
    print(event)
    print("---")

---GENERATE---
{'generate': {'messages': [AIMessage(content="As the Chief Product Officer, I believe the most suitable product modality for this task would be a combination of a Document and a Website. The Document (in Markdown format) would allow us to present the crawled content in a structured and readable manner, while the Website would enable us to showcase the images and their respective links in a visually appealing way. This approach would meet the customer's requirements while leveraging our existing product capabilities.")], 'env': {'DemandAnalysis': ''}}}
---
{'reflect': {'messages': [AIMessage(content="As the Chief Executive Officer, I agree with your assessment. A combination of a Document and a Website would be the most suitable product modality for this task. The Document would effectively capture the crawled content in a structured Markdown format, while the Website would elegantly display the images and their respective links. This approach aligns with our existing pro